<a href="https://colab.research.google.com/github/mazby45/mazzz/blob/main/SQLALCHEMY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect,Date,Float
from IPython.display import Markdown, display

In [ ]:
#Créer une connexion à la base de données jardin.db, si elle existe pas elle est créer
engine=create_engine('sqlite:///jardin.db')

In [ ]:
from sqlalchemy.sql.sqltypes import Boolean
meta = MetaData()
#creation de la Table clients
clients  = Table(
   'clients', meta,
   Column('Id_client', Integer, primary_key=True),
   Column('Nom', String),
   Column('Prenom', String),
   Column('Fidelite', Boolean)
)
meta.create_all(engine)

In [ ]:
#creation de la Table produits

produits  = Table(
   'produits', meta,
   Column('Id_produit', Integer, primary_key=True),
   Column('Designation', String),
   Column('Prix', Float)
)
meta.create_all(engine)

In [ ]:

#creation de la Table ventes
ventes  = Table(
   'ventes', meta,
   Column('Numero_vente', Integer, primary_key=True),
   Column('Date_vente', Date),
   Column('Id_produit', Integer, ForeignKey("produits.Id_produit")),
   Column('Id_client', Integer, ForeignKey("clients.Id_client"))
)
meta.create_all(engine)


In [ ]:
#verifions que les tables ont bien été crées
inspector = inspect(engine)
inspector.get_table_names()

['clients', 'produits', 'ventes']

In [ ]:
# creons des listes de clients pour notre table clients
liste_clients = [
    (1, "Balde","Saliou",True),
    (2, "Doe", "John", True),
    (3, "Smith", "Jane", False),
    (4, "Johnson", "Michael", True),
    (5, "Williams", "Emily", False),
    (6, "Brown", "David", True)
]

In [ ]:
# creons des listes de produits pour notre table produits
liste_produits = [
    (1, "Plante verte", 19.99),
    (2, "Graines de fleurs", 5.99),
    (3, "Terreau universel", 9.99),
    (4, "Arrosoir", 12.99),
    (5, "Pelle de jardin", 14.99)
]

In [ ]:
# creons des ventes pour notre table ventes
from datetime import date
liste_ventes = [
    (1, date(2023, 1, 1), 1, 1),
    (2, date(2023, 1, 2), 2, 2),
    (3, date(2023, 1, 3), 3, 3),
    (4, date(2023, 1, 1), 1, 1),
    (5, date(2023, 1, 2), 2, 2),
    (6, date(2023, 1, 3), 3, 3)
]

Nous allons maintenant remplir nos tables avec les listes crées precedemment

In [ ]:
#remplissons notre table clients
from sqlalchemy import insert
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            stmt = insert(clients).values(liste_clients)
            connection.execute(stmt)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [ ]:
#remplissons notre table produits
from sqlalchemy import insert
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            stmt = insert(produits).values(liste_produits)
            connection.execute(stmt)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [ ]:
#Créons des ventes
from sqlalchemy import insert
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            stmt = insert(ventes).values(liste_ventes)
            connection.execute(stmt)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [ ]:
#verifions que les entrées ont bien été creer
import pandas as pd
conn = engine.connect()
stmt = text("SELECT clients.Nom, clients.Prenom, SUM(produits.Prix) AS CA FROM ventes JOIN clients ON ventes.Id_client = clients.Id_client JOIN produits ON ventes.Id_produit = produits.Id_produit GROUP BY clients.Nom, clients.Prenom;")
#stmt contient notre requette SQL
result = conn.execute(stmt)
data=pd.DataFrame(result.fetchall()) #Pour obtenir le résultat de cette requête dans Python, on utilise la méthode fetchall associée à cet objet.

In [ ]:
#affichons le dataframe
display(data)

,Nom,Prenom,CA
0,Balde,Saliou,39.98
1,Doe,John,11.98
2,Smith,Jane,19.98


Bonus !! On peut ajouter dans notre requette SQL "WHERE clients.Fidelite=1" pour ne sélectionner que les clients qui sont inscris au programme de fidelité